Apply preprocessing - implement random forest, bagclassifier,XGBRFClassifier, SVM after applying word2vec, RF=52%, SVM=58%, Bagging = 58%, XGBRFClassifier =56%

In [ ]:
!pip install PyStemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#import libraries
from pandas import read_excel
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from ast import literal_eval
import pandas as pd
import numpy as np
import string 
import re
from nltk import word_tokenize
import json
nltk.download('punkt')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec
from gensim.models.word2vec import Word2Vec
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LinearRegression,SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRFClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#download dataset
file_name = 'Lab 2_icsme-questions-labeled.xlsx' # change it to the name of your excel file
df = read_excel(file_name)
df.head(3)

/usr/local/lib/python3.8/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,inline-comment-id,# Comment,Question,Redquestion,Low-level Label,Top-level Label
0,84326dd1_566c7146,1,is this what they intended? don't they really ...,is this what they intended?,request for confirmation,request
1,84326dd1_566c7146,2,is this what they intended? don't they really ...,don't they really want $(TARGET_OUT_DATA_NATIV...,surprise,attitudes and emotions
2,99d1f8e4_92b31cea,3,Don't we need to increment 'i' in the else cas...,Don't we need to increment 'i' in the else cas...,suggestion,suggestion


In [ ]:
#name of columns and shape of dataset
df.columns, df.shape

(Index(['inline-comment-id', '# Comment', 'Question', 'Redquestion',
        'Low-level Label', 'Top-level Label'],
       dtype='object'), (493, 6))

In [ ]:
#drop columns that do nto need
df = df.drop(columns=['inline-comment-id',"# Comment","Low-level Label"])

In [ ]:
#check null values
df.isnull().sum()

Question           0
Redquestion        0
Top-level Label    0
dtype: int64

In [ ]:
#check labels
df['Top-level Label'].value_counts()

request                   273
suggestion                164
attitudes and emotions     37
hypothetical scenario      13
rhetorical question         6
Name: Top-level Label, dtype: int64

In [ ]:
#lower case all the worrds
df.Redquestion  = df.Redquestion.str.lower()
df["Top-level Label"]  = df["Top-level Label"].str.lower()
df["Question"]  = df["Question"].str.lower()

# Split the input from the output
X = df[['Question']]
y = df['Top-level Label']
print(X.shape, y.shape, y)
#convert categorial to numerical
y= pd.factorize(y)[0]
#print(y)

(493, 1) (493,) 0                     request
1      attitudes and emotions
2                  suggestion
3                  suggestion
4      attitudes and emotions
                ...          
488                   request
489                   request
490                suggestion
491                suggestion
492                   request
Name: Top-level Label, Length: 493, dtype: object


In [ ]:
#remove stopwords
stop_words = set(stopwords.words('english'))

df['Question'] = df['Question'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df.head()

,Question,Redquestion,Top-level Label
0,intended? really want $(target_out_data_native...,is this what they intended?,request
1,intended? really want $(target_out_data_native...,don't they really want $(target_out_data_nativ...,attitudes and emotions
2,need increment 'i' else case avoid infinite loop?,don't we need to increment 'i' in the else cas...,suggestion
3,can't see anywhere set false. adjust single re...,should we just adjust the single reference in ...,suggestion
4,sure want include source file directly ? creat...,are you sure you want to include this source f...,attitudes and emotions


In [ ]:
#remove punctuation 
result = string.punctuation 
def remove_punctuations(text):
  result = string.punctuation 
  for punctuation in result:
      text = text.replace(punctuation, '')
  return text

df['Question'] = df['Question'].apply(remove_punctuations)
df.head()

,Question,Redquestion,Top-level Label
0,intended really want targetoutdatanativetest,is this what they intended?,request
1,intended really want targetoutdatanativetest,don't they really want $(target_out_data_nativ...,attitudes and emotions
2,need increment i else case avoid infinite loop,don't we need to increment 'i' in the else cas...,suggestion
3,cant see anywhere set false adjust single refe...,should we just adjust the single reference in ...,suggestion
4,sure want include source file directly create...,are you sure you want to include this source f...,attitudes and emotions


In [ ]:
#tokenize the words
def tokenize(column):
    tokens = nltk.word_tokenize(column)
    return [w for w in tokens if w.isalpha()]   

df['tokenized'] = df.apply(lambda x: tokenize(x['Question']), axis=1)
df[['tokenized']].head()

,tokenized
0,"[intended, really, want, targetoutdatanativetest]"
1,"[intended, really, want, targetoutdatanativetest]"
2,"[need, increment, i, else, case, avoid, infini..."
3,"[cant, see, anywhere, set, false, adjust, sing..."
4,"[sure, want, include, source, file, directly, ..."


In [ ]:
#stemming
ps = PorterStemmer()

df['stemmed'] = df['tokenized'].apply(lambda x: [ps.stem(y) for y in x])
df.head(2)

,Question,Redquestion,Top-level Label,tokenized,stemmed
0,intended really want targetoutdatanativetest,is this what they intended?,request,"[intended, really, want, targetoutdatanativetest]","[intend, realli, want, targetoutdatanativetest]"
1,intended really want targetoutdatanativetest,don't they really want $(target_out_data_nativ...,attitudes and emotions,"[intended, really, want, targetoutdatanativetest]","[intend, realli, want, targetoutdatanativetest]"


In [ ]:
#implement word2vec
word2vec = Word2Vec(df['stemmed'], min_count=1)

In [ ]:
word2vec = Word2Vec(df['stemmed'],min_count=1,sg=1)
word2vec.wv.vocab

{'intend': <gensim.models.keyedvectors.Vocab at 0x7fda8c6be190>,
 'realli': <gensim.models.keyedvectors.Vocab at 0x7fda8ea9c250>,
 'want': <gensim.models.keyedvectors.Vocab at 0x7fda8e87be20>,
 'targetoutdatanativetest': <gensim.models.keyedvectors.Vocab at 0x7fda8c4a2070>,
 'need': <gensim.models.keyedvectors.Vocab at 0x7fda8c4a20d0>,
 'increment': <gensim.models.keyedvectors.Vocab at 0x7fda8c4a2130>,
 'i': <gensim.models.keyedvectors.Vocab at 0x7fda8c4a2190>,
 'els': <gensim.models.keyedvectors.Vocab at 0x7fda8c4a21f0>,
 'case': <gensim.models.keyedvectors.Vocab at 0x7fda8c4a2250>,
 'avoid': <gensim.models.keyedvectors.Vocab at 0x7fda8c4a22b0>,
 'infinit': <gensim.models.keyedvectors.Vocab at 0x7fda8c4a2310>,
 'loop': <gensim.models.keyedvectors.Vocab at 0x7fda8c4a2370>,
 'cant': <gensim.models.keyedvectors.Vocab at 0x7fda8c4a23d0>,
 'see': <gensim.models.keyedvectors.Vocab at 0x7fda8c4a2430>,
 'anywher': <gensim.models.keyedvectors.Vocab at 0x7fda8c4a2490>,
 'set': <gensim.models.ke

In [ ]:
arrs = []
for sentense in tqdm.tqdm(df['stemmed']):
    sentense_vec = [word2vec.wv[i] for i in sentense if i in word2vec.wv.vocab]
    if not sentense_vec:
        sentense_vec.append(np.zeros(100))
    arr = np.array(sentense_vec)
    arr = arr[~np.isnan(arr).any(axis=1)]
    mean = np.mean(arr, axis=0)
    arrs.append(mean)

100%|██████████| 493/493 [00:00<00:00, 6564.06it/s]


In [ ]:
w2v_vectors = np.array(arrs)

In [ ]:
#change name of column
df['label'] = df["Top-level Label"]
df.head(2)

,Question,Redquestion,Top-level Label,tokenized,stemmed,label
0,intended really want targetoutdatanativetest,is this what they intended?,request,"[intended, really, want, targetoutdatanativetest]","[intend, realli, want, targetoutdatanativetest]",request
1,intended really want targetoutdatanativetest,don't they really want $(target_out_data_nativ...,attitudes and emotions,"[intended, really, want, targetoutdatanativetest]","[intend, realli, want, targetoutdatanativetest]",attitudes and emotions


In [ ]:
#implement random forest
y_train = y
rfc_word2vec = RandomForestClassifier()
rfc_word2vec.fit(w2v_vectors, y_train)  
precision = cross_val_score(rfc_word2vec, w2v_vectors, y_train, cv=10, scoring='precision_macro')
recall = cross_val_score(rfc_word2vec, w2v_vectors, y_train, cv=10, scoring='recall_macro')
f1 = cross_val_score(rfc_word2vec, w2v_vectors, y_train, cv=10, scoring='f1_macro')
accuracy = cross_val_score(rfc_word2vec, w2v_vectors, y_train, cv=10, scoring='accuracy')
print("%0.2f f1-score with a standard deviation of %0.2f" % (f1.mean(), f1.std()))
print("%0.2f recall with a standard deviation of %0.2f" % (recall.mean(), recall.std()))
print("%0.2f precision with a standard deviation of %0.2f" % (precision.mean(), precision.std()))
print("%0.2f accuracy with a standard deviation of %0.2f" % (accuracy.mean(), accuracy.std()))

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=10.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parame

0.20 f1-score with a standard deviation of 0.05
0.23 recall with a standard deviation of 0.04
0.23 precision with a standard deviation of 0.03
0.52 accuracy with a standard deviation of 0.04


In [ ]:
#implement SVM
y_train=y
svm_word2vec = SVC()
svm_word2vec.fit(w2v_vectors, y_train)  
precision = cross_val_score(svm_word2vec, w2v_vectors, y_train, cv=10, scoring='precision_macro')
recall = cross_val_score(svm_word2vec, w2v_vectors, y_train, cv=10, scoring='recall_macro')
f1 = cross_val_score(svm_word2vec, w2v_vectors, y_train, cv=10, scoring='f1_macro')
accuracy = cross_val_score(svm_word2vec, w2v_vectors, y_train, cv=10, scoring='accuracy')
print("%0.2f f1-score with a standard deviation of %0.2f" % (f1.mean(), f1.std()))
print("%0.2f recall with a standard deviation of %0.2f" % (recall.mean(), recall.std()))
print("%0.2f precision with a standard deviation of %0.2f" % (precision.mean(), precision.std()))
print("%0.2f accuracy with a standard deviation of %0.2f" % (accuracy.mean(), accuracy.std()))

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=10.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parame

0.20 f1-score with a standard deviation of 0.04
0.24 recall with a standard deviation of 0.04
0.28 precision with a standard deviation of 0.09
0.58 accuracy with a standard deviation of 0.03


In [ ]:
#implementBaggingClassifier
bag_word2vec = BaggingClassifier(base_estimator=SVC(),n_estimators=10, random_state=0).fit(w2v_vectors, y_train) 
y_train=y
precision = cross_val_score(bag_word2vec, w2v_vectors, y_train, cv=10, scoring='precision_macro')
recall = cross_val_score(bag_word2vec, w2v_vectors, y_train, cv=10, scoring='recall_macro')
f1 = cross_val_score(bag_word2vec, w2v_vectors, y_train, cv=10, scoring='f1_macro')
accuracy = cross_val_score(bag_word2vec, w2v_vectors, y_train, cv=10, scoring='accuracy')
print("%0.2f f1-score with a standard deviation of %0.2f" % (f1.mean(), f1.std()))
print("%0.2f recall with a standard deviation of %0.2f" % (recall.mean(), recall.std()))
print("%0.2f precision with a standard deviation of %0.2f" % (precision.mean(), precision.std()))
print("%0.2f accuracy with a standard deviation of %0.2f" % (accuracy.mean(), accuracy.std()))

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=10.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in v

0.20 f1-score with a standard deviation of 0.04
0.24 recall with a standard deviation of 0.04
0.28 precision with a standard deviation of 0.09
0.58 accuracy with a standard deviation of 0.03


In [ ]:
#implement XGBRFClassifier

y_train = y
model_XGBRF = XGBRFClassifier(n_estimators=100, subsample=0.9, colsample_bynode=0.2)
model_XGBRF.fit(w2v_vectors, y_train)  

precision = cross_val_score(model_XGBRF, w2v_vectors, y_train, cv=10, scoring='precision_macro')
recall = cross_val_score(model_XGBRF, w2v_vectors, y_train, cv=10, scoring='recall_macro')
f1 = cross_val_score(model_XGBRF, w2v_vectors, y_train, cv=10, scoring='f1_macro')
accuracy = cross_val_score(model_XGBRF, w2v_vectors, y_train, cv=10, scoring='accuracy')
print("%0.2f f1-score with a standard deviation of %0.2f" % (f1.mean(), f1.std()))
print("%0.2f recall with a standard deviation of %0.2f" % (recall.mean(), recall.std()))
print("%0.2f precision with a standard deviation of %0.2f" % (precision.mean(), precision.std()))
print("%0.2f accuracy with a standard deviation of %0.2f" % (accuracy.mean(), accuracy.std()))

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=10.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parame

0.22 f1-score with a standard deviation of 0.05
0.24 recall with a standard deviation of 0.04
0.24 precision with a standard deviation of 0.04
0.56 accuracy with a standard deviation of 0.04
